# Project 1 - object removal

## Imports

In [ ]:
import cv2
import numpy as np
import PIL
from PIL import Image 

In [ ]:

def imshow(img):
    img = img.clip(0, 255).astype("uint8")
    if img.ndim == 3:
        if img.shape[2] == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    display(PIL.Image.fromarray(img))

In [ ]:
img1 = cv2.imread('DJI_20231031123737_0028_W.JPG')
img2 = cv2.imread('DJI_20231031123813_0040_W.JPG')
img3 = cv2.imread('DJI_20231031123834_0047_W.JPG')
img4 = cv2.imread('fnal1.jpg')
img5 = cv2.imread('fnal2.jpg')

# Pixel average and median

In [ ]:
stacked_img = np.stack((img1, img2, img3))
average_img = np.average(stacked_img, axis=0)
median_img = np.median(stacked_img, axis=0)
print("average:")
imshow(average_img)
print("median:")
imshow(median_img)

In [ ]:
def open(img):
    return cv2.dilate(cv2.erode(img, np.ones((4,4))), np.ones((4,4)))

def close(img):
    return cv2.erode(cv2.dilate(img, np.ones((3,3))), np.ones((3,3)))

def proper_open(img):
    to_compare = close(open(close(img)))
    stack = np.stack((img, to_compare))
    return np.min(stack, axis=0)

def proper_close(img):
    to_compare = open(close(open(img)))
    stack = np.stack((img, to_compare))
    return np.max(stack, axis=0)

def automedian_filter(img):
    to_compare = proper_open(img)
    stack = np.stack((img, to_compare))
    return np.max(stack, axis=0)


In [ ]:
img1_grayscale = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2_grayscale = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img3_grayscale = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

img1_sobel = np.abs(cv2.Sobel(img1_grayscale, cv2.CV_64F, 1, 0, ksize=3))
img2_sobel = np.abs(cv2.Sobel(img2_grayscale, cv2.CV_64F, 1, 0, ksize=3))
img3_sobel = np.abs(cv2.Sobel(img3_grayscale, cv2.CV_64F, 1, 0, ksize=3))

stacked_sobel = np.stack((img1_sobel, img2_sobel, img3_sobel))
sobel_min = np.min(stacked_sobel, axis=0)


In [ ]:

sobel_min[sobel_min < 2] = 0

reduced = img1_sobel - 5 * sobel_min
open_result1 = (open(reduced))
binarized1 = (open_result1 > 0).astype(np.uint8) * 255


reduced = img2_sobel - 5 * sobel_min
open_result2 = (open(reduced))
binarized2 = (open_result2 > 0).astype(np.uint8) * 255

reduced = img3_sobel - 5 * sobel_min
open_result3 = (open(reduced))
binarized3 = (open_result3 > 0).astype(np.uint8) * 255

for i in range(3):
    binarized1 = cv2.dilate(binarized1, np.ones((20,20)))
    binarized2 = cv2.dilate(binarized2, np.ones((20,20)))
    binarized3 = cv2.dilate(binarized3, np.ones((20,20)))



In [ ]:

images=[]
for img, mask in zip([img1,img2,img3], [binarized1,binarized2, binarized3]):
    images.append(cv2.inpaint(img, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA))
stacked_img=np.stack(images)
for x in images:
    imshow(x)
inpainted_image = np.median(stacked_img, axis=0)
imshow(inpainted_image)

In [ ]:
def Automedian(img, s):
  return np.maximum(cv2.morphologyEx(cv2.morphologyEx(cv2.morphologyEx(img.copy(), cv2.MORPH_OPEN, s), cv2.MORPH_CLOSE, s), cv2.MORPH_OPEN, s),np.minimum(cv2.morphologyEx(cv2.morphologyEx(cv2.morphologyEx(img.copy(), cv2.MORPH_CLOSE, s), cv2.MORPH_OPEN, s), cv2.MORPH_CLOSE, s), img))


In [ ]:
def open(img,s):
    return cv2.dilate(cv2.erode(img, s), s)

def close(img,s):
    return cv2.erode(cv2.dilate(img, s), s)

def proper_open(img,s):
    to_compare = close(open(close(img,s),s),s)
    stack = np.stack((img, to_compare))
    return np.min(stack, axis=0)

def proper_close(img,s):
    to_compare = open(close(open(img,s),s),s)
    stack = np.stack((img, to_compare))
    return np.max(stack, axis=0)


In [ ]:



    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    gray3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

    stacked_img = np.stack((img1, img2, img3))

    median_img = np.median(stacked_img, axis=0).astype(np.uint8)
    gray4=cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY)
    # Compute absolute differences between consecutive frames
    diff1 = cv2.absdiff(gray1, gray4)
    diff2 = cv2.absdiff(gray2, gray4)
    diff3 = cv2.absdiff(gray3, gray4)
    # Threshold the differences to get a binary mask
    _, thresh1 = cv2.threshold(diff1, 70, 255, cv2.THRESH_BINARY)
    _, thresh2 = cv2.threshold(diff2, 70, 255, cv2.THRESH_BINARY)
    _, thresh3 = cv2.threshold(diff3, 70, 255, cv2.THRESH_BINARY)
    s=np.ones((5,5))
    
    
 

    # Combine the binary masks
    combined_mask = cv2.bitwise_or(cv2.bitwise_or(thresh1, thresh2), thresh3)
        # Apply morphological operations to improve the mask
    kernel = np.ones((3,3), np.uint8)

    combined_mask = Automedian(combined_mask, kernel)

    combined_mask=proper_open(combined_mask, np.ones((3,3)))
    combined_mask=Automedian(combined_mask, np.ones((21,21)))
    
    imshow(combined_mask)
   # Inpainting to fill in the gaps
    inpainted_image1 = cv2.inpaint(median_img.astype(np.uint8), combined_mask, inpaintRadius=31, flags=cv2.INPAINT_TELEA)


    img=inpainted_image1
    imshow(img)

In [ ]:
kernel = np.ones((10,10))
imshow(thresh1)
imshow(thresh2)
imshow(thresh3)

filled_images = []
filled_images2 = []

for img_ in [thresh1, thresh2, thresh3]:
    img_t = close(img_, kernel)
    contours, _ = cv2.findContours(img_t, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    filled_image = np.zeros_like(img_t)

    # Fill concave parts in the contours
    for contour in contours:
        cv2.drawContours(filled_image, [contour], 0, 255, -1)

    imshow(filled_image)
    filled_images.append(filled_image)
    filled_images2.append(filled_image)

for i1, im1 in enumerate(filled_images):
    for i2, im2 in enumerate(filled_images2):
        if i1 != i2:
            pass
            #im1[im2.astype(np.bool_)] = 0


for im in filled_images:
    pass
    imshow(im)




In [ ]:
filtered_images = list()
filtered_images2 = list()

for image in filled_images:

    # Apply connected component analysis
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=8)

    # Set a threshold for region size
    threshold_size = 100  # Adjust this threshold as needed

    # Iterate through the regions and filter based on size threshold
    filtered_regions = []
    for i in range(1, num_labels):  # Start from 1 to skip background (label 0)
        region_size = stats[i, cv2.CC_STAT_AREA]
        if region_size > threshold_size:
            filtered_regions.append(labels == i)

    # Visualize the filtered regions
    filtered_image = np.zeros_like(image)
    for region in filtered_regions:
        filtered_image |= region.astype(np.uint8) * 255


    # Find extreme points for each filtered region
    extreme_points_list = []
    for region in filtered_regions:
        contours, _ = cv2.findContours(region.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            contour = contours[0]
            extreme_points = np.squeeze(contour, axis=1)
            min_x = np.min(extreme_points[:, 0])
            min_y = np.min(extreme_points[:, 1])
            max_x = np.max(extreme_points[:, 0])
            max_y = np.max(extreme_points[:, 1])

            filtered_image[min_y:max_y, min_x:max_x] = 255

    # Display the resulting image
    filtered_images.append(filtered_image)
    filtered_images2.append(filtered_image)

mask1 = filtered_images[0].astype(np.bool_)
mask2 = filtered_images[1].astype(np.bool_)
mask3 = filtered_images[2].astype(np.bool_)

    
mask12 = mask1 & mask2
mask13 = mask1 & mask3
mask23 = mask2 & mask3

for i1, im1 in enumerate(filtered_images):
    for i2, im2 in enumerate([mask23, mask13, mask12]):
        if i1 != i2:
            pass
            im1[im2.astype(np.bool_)] = 0



In [ ]:
for img, binarized in zip([img1, img2, img3], filled_images):
    stack = np.stack((img, np.repeat(binarized[:,: ,np.newaxis], 3, axis=2)))

    imshow(np.average(stack, axis=0))

In [ ]:

    img1_modified = img1.copy()
    img2_modified = img2.copy()
    img3_modified = img3.copy()

    img12_modified = np.stack((img1, img2))
    img13_modified = np.stack((img1, img3))
    img23_modified = np.stack((img2, img3))
    img12_modified = np.median(img12_modified, axis=0)
    img13_modified = np.median(img13_modified, axis=0)
    img23_modified = np.median(img23_modified, axis=0)

    img123_modified = np.stack((img1, img2, img3))
    img123_modified = np.median(img123_modified, axis=0)

    mask1 = filled_images[0].astype(np.bool_)
    mask2 = filled_images[1].astype(np.bool_)
    mask3 = filled_images[2].astype(np.bool_)

    
    mask12 = mask1 & mask2
    mask13 = mask1 & mask3
    mask23 = mask2 & mask3
    
    mask123 = mask12 & mask3

    img1_modified[mask1 | ~mask2 | ~mask3] = 0
    img2_modified[mask2 | ~mask1 | ~mask3] = 0
    img3_modified[mask3 | ~mask1 | ~mask2] = 0
    img12_modified[mask1 | mask2 | ~mask3] = 0
    img13_modified[mask1 | ~mask2 | mask3] = 0
    img23_modified[~mask1 | mask2 | mask3] = 0

    img123_modified[(mask1 | mask2 | mask3) & ~mask123] = 0

    final = np.stack((img1_modified, img2_modified, img3_modified, img12_modified, img13_modified, img23_modified, img123_modified))
    final = np.sum(final, axis=0)
    imshow(img12_modified)
    imshow(img13_modified)
    imshow(img23_modified)
    imshow(final)

    final = cv2.inpaint(final.astype(np.uint8), mask123.astype(np.uint8), 20, cv2.INPAINT_NS)
    imshow(final)
    cv2.imwrite('fnal2.jpg', final)

In [ ]:
img1 = cv2.imread('DJI_20231031123737_0028_W.JPG')
img2 = cv2.imread('DJI_20231031123813_0040_W.JPG')
img3 = cv2.imread('DJI_20231031123834_0047_W.JPG')
img4 = cv2.imread('fnal1.jpg')
img5 = cv2.imread('fnal2.jpg')

In [ ]:
stacked_img = np.stack((img1, img2, img3, img4, img5))
average_img = np.average(stacked_img, axis=0)
median_img = np.median(stacked_img, axis=0)
print("average:")
imshow(average_img)
print("median:")
imshow(median_img)